#### Bibliotecas

In [1]:
import sys
import tweepy as tw
import pandas as pd

if '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco' not in sys.path:
    sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco')
if '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco/maria_bonita/conjunto_dados' not in sys.path:
    sys.path.insert(0, '/Users/will/Google Drive/workspace/Projetos/projeto_cangaco/maria_bonita/conjunto_dados')

from utilidades.autenticacao import autenticar
import maria_bonita.conjunto_dados.utilidades.captura.StreamListenerPersonalizado as slp
import maria_bonita.conjunto_dados.utilidades.captura.API_Personalizada as ap
from maria_bonita.conjunto_dados.utilidades.captura.chaves_busca import MISOGINIA
import maria_bonita.conjunto_dados.utilidades.captura.ferramentas_conjunto_dados as fcd
import maria_bonita.conjunto_dados.utilidades.pre_processamento.novas_variaveis as nv

#### Parâmetros

In [2]:
# DEFINIR PARÂMETROS !!!
chaves_escolhidas = [MISOGINIA]

chaves_busca, quantidade_tweets, com_retweets, \
idiomas, caminho_dataset = fcd.parametros(chaves_busca=chaves_escolhidas,
                                          quantidade_tweets=30)

#### Rotina

In [3]:
auth = autenticar()
lista_tweets = []

minha_stream = None
minha_stream = tw.Stream(auth=auth,
                         listener=slp.StreamListenerPersonalizado(lista_resultados=lista_tweets,
                                                                  quantidade_tweets=quantidade_tweets,
                                                                  com_retweets=com_retweets))

minha_stream.filter(languages=idiomas,
                    track=chaves_busca)

1 -----------------------------------------------------------------------
Usuário: byvantetaetae
Texto: eu sou burra, não sei usar o tt, vou bater a cabeça na parede
------------------------------------------------------------------------

2 -----------------------------------------------------------------------
Usuário: B50tonsdprazer
Texto: @Anabastosex @Beatrizvirgem Delícia de ladinho socava bem gostoso.
------------------------------------------------------------------------

3 -----------------------------------------------------------------------
Usuário: allforportilla
Texto: essa maluca me dando block que sabor de mel
------------------------------------------------------------------------

4 -----------------------------------------------------------------------
Usuário: Samueld92476695
Texto: @Lamamiuno1 Delícia
------------------------------------------------------------------------

5 -----------------------------------------------------------------------
Usuário: beacasag

KeyboardInterrupt: 

#### Visualização

In [4]:
conjunto_dados = pd.DataFrame(lista_tweets)
conjunto_dados.head()

,criado_em,retweet,tweet_id,tweet_url,tweet_texto,tweet_hashtags,tweet_urls_externos,tweet_usuarios_mencionados,tweet_plataforma_origem,tweet_em_resposta,...,usuario_verificado,usuario_localizacao,usuario_descricao,usuario_qtd_seguidores,usuario_qtd_amigos,usuario_qtd_tweets,usuario_url_imagem_perfil,usuario_url_imagem_capa,usuario_criado_em,usuario_json
0,2020-12-05 15:00:59,False,1335237740842135553,https://twitter.com/byvantetaetae/status/13352...,"eu sou burra, não sei usar o tt, vou bater a c...",[],[],[],Android,None,...,False,태태,For Vante.♡\n\nKim Taehyung my love; Jeongguk ...,806,1080,88238,,,2020-03-20 22:58:04,"{'id': 1241136912519266305, 'id_str': '1241136..."
1,2020-12-05 15:00:59,False,1335237742209413124,https://twitter.com/B50tonsdprazer/status/1335...,@Anabastosex @Beatrizvirgem Delícia de ladinho...,[],[],"[{'screen_name': 'Anabastosex', 'id_str': '133...",Android,"{'resposta_ao_tweet': '1335188942229041153', '...",...,False,None,nv aki.chama no ADM.,216,303,1190,,,2020-07-18 13:06:41,"{'id': 1284474125814243328, 'id_str': '1284474..."
2,2020-12-05 15:01:00,False,1335237742880567299,https://twitter.com/allforportilla/status/1335...,essa maluca me dando block que sabor de mel,[],[],[],iPhone,None,...,False,None,here to support anahi portilla and alfonso her...,1622,801,97793,,,2012-02-11 23:31:24,"{'id': 489839672, 'id_str': '489839672', 'name..."
3,2020-12-05 15:01:00,False,1335237743169982467,https://twitter.com/Samueld92476695/status/133...,@Lamamiuno1 Delícia,[],[],"[{'screen_name': 'Lamamiuno1', 'id_str': '9815...",Android,"{'resposta_ao_tweet': '1321493450311766016', '...",...,False,None,legal,157,4541,642,,,2020-01-27 10:27:37,"{'id': 1221741503225135104, 'id_str': '1221741..."
4,2020-12-05 15:01:00,False,1335237743937511426,https://twitter.com/beacasagrande/status/13352...,"vendo ela de camisa do flamengo, DO FLAMENGO",[],[],[],Android,None,...,False,sc,"meio historiadora, mas não sei todas as datas",2325,1302,100606,,,2011-08-04 16:26:41,"{'id': 348550695, 'id_str': '348550695', 'name..."


#### Consolidar dados

In [5]:
if not "conjunto_dados" in locals():
    conjunto_dados = pd.DataFrame(lista_tweets)
conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')

#### Pré-processamento

In [6]:
# Com conexão no end-point

if not "conjunto_dados" in locals():
    conjunto_dados = pd.read_csv(caminho_dataset, sep=';')

API = ap.API()
conjunto_dados['tweet_usuarios_mencionados'] = conjunto_dados.apply(lambda x: fcd.relacionamento_com_mencionados(API=API,
                                                                                                                 usuario_id_origem=x["usuario_id"],
                                                                                                                 mencionados=x['tweet_usuarios_mencionados']),
                                                                    axis=1)

API = ap.API()
conjunto_dados['tweet_em_resposta'] = conjunto_dados.apply(lambda x: fcd.relacionamento_com_respondido(API=API,
                                                                                                       usuario_id_origem=x["usuario_id"],
                                                                                                       mencionados=x['tweet_usuarios_mencionados'],
                                                                                                       em_resposta=x['tweet_em_resposta']), axis=1)

conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')


Chamadas API: 1/179

Chamadas API: 2/179

Chamadas API: 3/179

Chamadas API: 4/179

Chamadas API: 5/179

Chamadas API: 6/179

Chamadas API: 7/179

Chamadas API: 8/179

Chamadas API: 9/179

Chamadas API: 10/179

Chamadas API: 11/179

Chamadas API: 12/179

Chamadas API: 13/179

Chamadas API: 14/179


In [7]:
# Sem conexão no end-point

if not "conjunto_dados" in locals():
    conjunto_dados = pd.read_csv(caminho_dataset, sep=';')
    
conjunto_dados['relacionamento_com_mencionados'] = conjunto_dados['tweet_usuarios_mencionados'].apply(lambda x: nv.relacionamento_com_mencionados(mencionados=x))

conjunto_dados['relacionamento_com_respondido'] = conjunto_dados['tweet_em_resposta'].apply(lambda x: nv.relacionamento_com_respondido(em_resposta=x))

conjunto_dados['texto_original'] = conjunto_dados['tweet_texto']

conjunto_dados['tokens'] = conjunto_dados.apply(lambda x: nv.tokenizar(tweet_texto=x['tweet_texto'],
                                                                       chaves_busca=chaves_escolhidas,
                                                                       mencionados=x['tweet_usuarios_mencionados']),
                                                axis=1)

conjunto_dados['possivel_crime'] = None

conjunto_dados.to_csv(caminho_dataset, sep=';', index=False, encoding='utf-8-sig')